# LDA (Octis) - Yahoo! Answers
## ECS271 
## Kay Royo


In [1]:
!pip install octis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 129 kB 7.8 MB/s 
     |████████████████████████████████| 85 kB 4.5 MB/s 
     |████████████████████████████████| 170 kB 45.6 MB/s 
     |████████████████████████████████| 24.1 MB 1.8 MB/s 
     |████████████████████████████████| 24.9 MB 1.4 MB/s 
     |████████████████████████████████| 16.5 MB 45.4 MB/s 
     |████████████████████████████████| 100 kB 9.5 MB/s 
     |████████████████████████████████| 5.8 MB 44.5 MB/s 
     |████████████████████████████████| 1.3 MB 41.8 MB/s 
     |████████████████████████████████| 182 kB 63.5 MB/s 
     |████████████████████████████████| 7.6 MB 28.2 MB/s 
  Created wheel for libsvm: filename=libsvm-3.23.0.4-cp38-cp38-linux_x86_64.whl size=233360 sha256=d37beec87fd6c12ef84a200ed2fe670d349ddd00cd3b5678f953b251cbd3166b
  Stored in directory: /root/.cache/pip/wheels/41/00/e7/b328c0f5e4bbb9ac8ba5a72e56b1749be63dc1ab1c9321

In [2]:
import pandas as pd

In [3]:
from octis.models.LDA import LDA
from sentence_transformers import SentenceTransformer
from octis.evaluation_metrics.diversity_metrics import TopicDiversity
from octis.evaluation_metrics.coherence_metrics import Coherence
from octis.dataset.dataset import Dataset
import os
import string
from octis.preprocessing.preprocessing import Preprocessing
os.chdir(os.path.pardir)

In [4]:
dt = pd.read_csv('../content/yahoo_cleaned_tr.csv')
df=list(dt['Text'])
lbl=list(dt['Class_Name']) 

In [5]:
with open('yahoo_class.txt', 'w') as f:
    for line in lbl:
        f.write(line)
        f.write('\n')

In [6]:

with open('yahoo_text.txt', 'w') as f:
    for line in df:
        f.write(line)
        f.write('\n')

In [7]:
# Initialize preprocessing
preprocessor = Preprocessing(vocabulary=None, max_features=None,
                             remove_punctuation=True, punctuation=string.punctuation,
                             lemmatize=True, stopword_list='english',
                             min_chars=1, min_words_docs=0)
# preprocess
dataset = preprocessor.preprocess_dataset(documents_path=r'../yahoo_text.txt', labels_path=r'../yahoo_class.txt')

# save the preprocessed dataset
dataset.save('yahoo_dt')

created vocab
91376


In [8]:
dataset= Dataset()
dataset.load_custom_dataset_from_folder("../yahoo_dt")

In [9]:
# Create Model
lda_model_oct = LDA(num_topics=10, 
            passes=4,
            random_state=None)

In [10]:
%%time
# Train the model using default partitioning choice 
output = lda_model_oct.train_model(dataset)

print(*list(output.keys()), sep="\n") # Print the output identifiers

topic-word-matrix
topics
topic-document-matrix
test-topic-document-matrix
CPU times: user 1min 18s, sys: 854 ms, total: 1min 19s
Wall time: 1min 19s


# Topic diversity

In [14]:
metric = TopicDiversity(topk=10) # Initialize metric
topic_diversity_score = metric.score(output) # Compute score of the metric
topic_diversity_score

0.9

# Coherence scores 

In [15]:
metric = Coherence(measure='c_v') # Initialize metric
c_v_score = metric.score(output) # Compute score of the metric
c_v_score

0.5665267029623002

In [16]:
metric = Coherence(measure='u_mass') # Initialize metric
u_mass_score = metric.score(output) # Compute score of the metric
u_mass_score

-2.1768208879082978

In [17]:
metric = Coherence(measure='c_uci') # Initialize metric
c_uci_score = metric.score(output) # Compute score of the metric
c_uci_score

0.10755100751453792

In [18]:
metric = Coherence(measure='c_npmi') # Initialize metric
c_npmi_score = metric.score(output) # Compute score of the metric
c_npmi_score

0.05233463124459341